In [1]:
import dagshub
import mlflow
mlflow.set_tracking_uri("https://dagshub.com/aryan0147/Capstone-Project-2.mlflow")

dagshub.init(repo_owner='aryan0147', repo_name='Capstone-Project-2', mlflow=True)

Accessing as aryan0147

Initialized MLflow to track repo "aryan0147/Capstone-Project-2"

Repository aryan0147/Capstone-Project-2 initialized!

In [2]:
# Set or create an experiment
mlflow.set_experiment("ML-Algos")

<Experiment: artifact_location='mlflow-artifacts:/d43192cc72bb48deb03d07938721755b', creation_time=1739197271560, experiment_id='12', last_update_time=1739197271560, lifecycle_stage='active', name='ML-Algos', tags={}>

In [3]:
import optuna
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [4]:
df = pd.read_csv('../reddit_preprocessing.csv').dropna()
df.shape

(36662, 2)

In [5]:
import mlflow
import optuna
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, classification_report

# Step 1: Remap the class labels from [-1, 0, 1] to [2, 0, 1]
df['category'] = df['category'].map({-1: 2, 0: 0, 1: 1})

# Step 2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

ngram_range = (1, 3)  # Trigram setting
max_features = 10000  # Set max_features to 1000 for TF-IDF

# **Step 3: Train-test split BEFORE any class balancing (Prevents Data Leakage)**
X_train, X_test, y_train, y_test = train_test_split(
    df['clean_comment'], df['category'], test_size=0.2, random_state=42, stratify=df['category']
)

# **Step 4: Fit TF-IDF only on training data**
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X_train_vec = vectorizer.fit_transform(X_train)  # Fit only on training data
X_test_vec = vectorizer.transform(X_test)  # Transform test data without refitting

# **Step 5: Compute Class Weights Instead of SMOTE**
unique_classes, class_counts = np.unique(y_train, return_counts=True)
class_weights = class_counts.sum() / (len(unique_classes) * class_counts)  # Compute scale_pos_weight
class_weight_dict = {cls: weight for cls, weight in zip(unique_classes, class_weights)}

# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        mlflow.set_tag("mlflow.runName", f"{model_name}_ClassWeights_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)

        # Make predictions
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")

# Step 6: Optuna objective function for XGBoost
def objective_xgboost(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)
    max_depth = trial.suggest_int('max_depth', 3, 10)

    # Compute class weight for XGBoost using scale_pos_weight
    scale_pos_weight = class_weights[1]  # Assuming class 1 is the minority class

    model = XGBClassifier(
        n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth,
        scale_pos_weight=scale_pos_weight, random_state=42
    )

    # Fit and evaluate the model
    model.fit(X_train_vec, y_train)
    y_pred = model.predict(X_test_vec)
    return accuracy_score(y_test, y_pred)

# Step 7: Run Optuna for XGBoost, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_xgboost, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = XGBClassifier(
        n_estimators=best_params['n_estimators'], learning_rate=best_params['learning_rate'],
        max_depth=best_params['max_depth'], scale_pos_weight=class_weights[1], random_state=42
    )

    # Log the best model with MLflow
    log_mlflow("XGBoost", best_model, X_train_vec, X_test_vec, y_train, y_test)

# Run the experiment for XGBoost
run_optuna_experiment()


[I 2025-02-10 20:23:23,038] A new study created in memory with name: no-name-a5cc22e1-0a17-47af-8fbe-ca2a5e483a90
c:\Users\Aryan\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [20:23:23] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "scale_pos_weight" } are not used.

  warnings.warn(smsg, UserWarning)
[I 2025-02-10 20:25:34,383] Trial 0 finished with value: 0.6532115096140734 and parameters: {'n_estimators': 186, 'learning_rate': 0.0007780142901083071, 'max_depth': 10}. Best is trial 0 with value: 0.6532115096140734.
c:\Users\Aryan\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [20:25:34] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "scale_pos_weight" } are not used.

  warnings.warn(smsg, UserWarning)
[I 2025-02-10 20:25:42,409] Tr

🏃 View run XGBoost_ClassWeights_TFIDF_Trigrams at: https://dagshub.com/aryan0147/Capstone-Project-2.mlflow/#/experiments/12/runs/8295bfb8ec45433bb2758802f5fdcf04
🧪 View experiment at: https://dagshub.com/aryan0147/Capstone-Project-2.mlflow/#/experiments/12
